<a href="https://colab.research.google.com/github/tunmed38/BotonBinance/blob/master/TA%2BRB_3110.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ติดตั้ง Module ที่จำเป็นสำหรับการเขียนโปรแกรม
!pip install ccxt
!pip install pandas
!pip install pandas_ta
!pip install parinya



## Settings 
import ccxt
import parinya
from parinya import LINE
import threading

line = LINE('jcgqm9ptQZ2GsTvZ9mLVScxpVanRdHJTV6dpxCBVUao')

# เปลี่ยน API เปลียน ข้อมูลกันด้วยนะครับ
apiKey    = "GHamJL0ZP3rYSnT0G7PMzHAjuSZmwiUCF0KegCYMYT2pMXxYZ1rdXc9Xkpj3Jmaq" 
secret    = "tbkkkYYnxBZcay0WmgeKfsEs7v3fDiw9t8NqoVhEMXUUM2VIz6ndNdAY72mcvOUA" 
password  = "" 
Account_name  = "TUNMED38" 
EMA_fast_set    = 8
EMA_slow_set    = 10

Pair_trade         = "ETH/USDT" # สินค้าที่จะเทรด
timeframe     = "15m" # ช่วงเวลาของกราฟที่จะดึงข้อมูล
limit         = 999 # ให้ดึงข้อมูลมากี่แถว ย้อนหลังกี่แทง
Unit_trade    = 0.003 # ปริมาณการเทรด

############################################################

exchange = ccxt.binance  ({'apiKey' : apiKey ,'secret' : secret ,'password' : password ,'enableRateLimit': True})
# Sub Account Check
if Account_name == "" :
  print("\n""Account Name - This is Main Account",': Broker - ',exchange)     
else:
  print( "\n"'Account Name - ',Account_name,': Broker - ',exchange)
  exchange.headers = {'binance-SUBACCOUNT': Account_name,}


  pair_trade = 'ETH/USDT'          #@param {type:"string"}
  Asset_RB   = 'ETH'              #@param {type:"string"}

  Rebalance_mark =             1000 #@param 
  Rebalance_percent = 1           #@param 
  

def start_mix ():
  import pandas as pd
  # เรียก ข้อมูลราคา ของสินค้ามาไว้ในรูปแบบ ตาราง 
  df_ohlcv              = exchange.fetch_ohlcv(Pair_trade ,timeframe=timeframe,limit=limit)
  df_ohlcv              = pd.DataFrame(df_ohlcv, columns =['datetime', 'open','high','low','close','volume'])
  df_ohlcv['datetime']  = pd.to_datetime(df_ohlcv['datetime'], unit='ms')

  import pandas_ta as ta

  EMA_fast_set = 8
  EMA_slow_set = 10

  EMA_fast         = df_ohlcv.ta.ema(EMA_fast_set)
  
  EMA_slow         = df_ohlcv.ta.ema(EMA_slow_set)
  df_ohlcv         = pd.concat([df_ohlcv,EMA_fast,EMA_slow], axis=1)
  

  Rebalance_mark = 1000
  
  

  #EMA Cross
  count = len(df_ohlcv)

  EMA_fast_A = df_ohlcv ['EMA_8'][count-2]
  EMA_fast_B = df_ohlcv ['EMA_8'][count-3]
  print("EMA_fast_A = ",EMA_fast_A)
  print("EMA_fast_B = ",EMA_fast_B)

  EMA_slow_A = df_ohlcv ['EMA_10'][count-2]
  EMA_slow_B = df_ohlcv ['EMA_10'][count-3]
  print("EMA_slow_A = ",EMA_slow_A)
  print("EMA_slow_B = ",EMA_slow_B)


  Get_balance = exchange.fetch_balance()
  Asset_01 = Get_balance [Asset_RB] ['total']
  print("Asset 01 = " , Asset_01,Asset_RB)

  get_price     = exchange.fetch_ticker(pair_trade) 
  Average_price = (get_price ['bid'] + get_price ['ask'] )/2
  print("Asset 01 = " , Asset_01*Average_price)
  print("Asset Rebalance_mark = " , Rebalance_mark)

  # มูลค่า ของ Asset
  Asset_01_Value = Asset_01 * Average_price
  print("Asset_01_Value = " ,Asset_01_Value)
  Rebalance_mark_temp = Unit_trade*Average_price

  # Signal and Trend
  Signal = "none"
  Trend = "none"

  if EMA_fast_A > EMA_slow_A :
    Trend = "Up_trend"
  elif EMA_fast_A < EMA_slow_A :
    Trend = "Down_trend"

  if EMA_fast_A > EMA_slow_A and EMA_fast_B < EMA_slow_B:
    Signal = "Buy_Signal"
  elif EMA_fast_A < EMA_slow_A and EMA_fast_B > EMA_slow_B:
    Signal = "Sell_Signal"

  print("\n""####### Signal and Trend #####")
  print("Trend  = ",Trend)
  print("Signal = ",Signal)

  print("\n""####### Trade Status #####")
  

  

  
  if Signal  == "Buy_Signal" and Asset_01_Value <= 1300:
      print("BUY-Trade")
  
      exchange.create_order( Pair_trade ,'market','buy',Unit_trade )
      line.sendtext('BUY-Trade on EMA Cross Over')
      line.sendtext(Asset_01_Value)
      line.sendtext(Average_price)
      
      
      
  elif Trend == "Up_trend" :
      Rebalance_mark_buy = 1300
      if Asset_01_Value > (Rebalance_mark_buy + (Rebalance_mark_buy*Rebalance_percent/100) ) :
        print("Asset_01_Value ",Asset_01_Value ,">", (Rebalance_mark_buy + (Rebalance_mark_buy*Rebalance_percent/100) ))
        print("SELL")
        diff_sell  = Asset_01_Value - Rebalance_mark_buy
        print(diff_sell)
        exchange.create_order( pair_trade ,'market','sell',(diff_sell/Average_price))
        line.sendtext('SELL-Trade on Rebalance')
        line.sendtext(Asset_01_Value)
        line.sendtext(Average_price)
        
      elif Asset_01_Value < (Rebalance_mark_buy - (Rebalance_mark_buy*Rebalance_percent/100) ) :
        print("Asset_01_Value ",Asset_01_Value ,"<", (Rebalance_mark_buy - (Rebalance_mark_buy*Rebalance_percent/100) ))
        print("Buy")
        diff_buy  = Rebalance_mark_buy - Asset_01_Value
        print(diff_buy)
        exchange.create_order( pair_trade ,'market','buy',(diff_buy/Average_price))
        line.sendtext('BUY-Trade on Rebalance')
        line.sendtext(Asset_01_Value)
        line.sendtext(Average_price)
        
      else :
        print("Non-Trade")
     

  elif Signal  == "Sell_Signal" and Asset_01_Value >= 1000 :
      print("SELL-Trade")
      exchange.create_order( Pair_trade ,'market','sell',Unit_trade )
      line.sendtext('SELL-Trade on EMA Cross Over')
      line.sendtext(Asset_01_Value)
      line.sendtext(Average_price)
      
    
  
  elif Trend == "Down_trend": 
      Rebalance_mark_sell = 1000
      if Asset_01_Value > (Rebalance_mark_sell + (Rebalance_mark_sell*(Rebalance_percent)/100) ) :
        print("Asset_01_Value ",Asset_01_Value ,">", (Rebalance_mark_sell + (Rebalance_mark_sell*Rebalance_percent/100) ))
        print("SELL")
        diff_sell  = Asset_01_Value - Rebalance_mark_sell
        print(diff_sell)
        exchange.create_order( pair_trade ,'market','sell',(diff_sell/Average_price))
        line.sendtext('SELL-Trade on Rebalance')
        line.sendtext(Asset_01_Value)
        line.sendtext(Average_price)
        

      elif Asset_01_Value < (Rebalance_mark_sell - (Rebalance_mark_sell*(Rebalance_percent)/100) ) :
        print("Asset_01_Value ",Asset_01_Value ,"<", (Rebalance_mark_sell - (Rebalance_mark_sell*Rebalance_percent/100) ))
        print("Buy")
        diff_buy  = Rebalance_mark_sell - Asset_01_Value
        print(diff_buy)
        exchange.create_order( pair_trade ,'market','buy',(diff_buy/Average_price))
        line.sendtext('BUY-Trade on Rebalance')
        line.sendtext(Asset_01_Value)
        line.sendtext(Average_price)
        
      else :
        print("Non-Trade")
       

  import time
  sleep = 30
  print("\n""Sleep",sleep,"sec.")
  time.sleep(sleep)  

def Loop() :
  while True:
    try :
      start_mix ()
    except Exception as e :
      line.sendtext('Mayday Mayday Autobot ERROR')
      import time
      sleep = 30
      print("\n""Sleep",sleep,"sec.")
      time.sleep(sleep)
  
  Loop()

Loop()



Account Name -  TUNMED38 : Broker -  Binance
EMA_fast_A =  4274.607331059232
EMA_fast_B =  4278.729425647584
EMA_slow_A =  4279.478823148235
EMA_slow_B =  4283.76745051451
Asset 01 =  0.2321262 ETH
Asset 01 =  982.9813372469999
Asset Rebalance_mark =  1000
Asset_01_Value =  982.9813372469999

####### Signal and Trend #####
Trend  =  Down_trend
Signal =  none

####### Trade Status #####
Asset_01_Value  982.9813372469999 < 990.0
Buy
17.018662753000058

Sleep 30 sec.
EMA_fast_A =  4274.607331059232
EMA_fast_B =  4278.729425647584
EMA_slow_A =  4279.478823148235
EMA_slow_B =  4283.76745051451
Asset 01 =  0.2361222 ETH
Asset 01 =  999.511175655
Asset Rebalance_mark =  1000
Asset_01_Value =  999.511175655

####### Signal and Trend #####
Trend  =  Down_trend
Signal =  none

####### Trade Status #####
Non-Trade

Sleep 30 sec.
EMA_fast_A =  4274.607331059232
EMA_fast_B =  4278.729425647584
EMA_slow_A =  4279.478823148235
EMA_slow_B =  4283.76745051451
Asset 01 =  0.2361222 ETH
Asset 01 =  1000

In [ ]:
pip freeze

absl-py==0.12.0
aiodns==3.0.0
aiohttp==3.7.4.post0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argcomplete==1.12.3
argon2-cffi==21.1.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==3.0.1
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
ccxt==1.59.72
certifi==2021.5.30
cffi==1.14.6
cftime==1.5.1
chardet==3.0.4
charset-normalizer==2.0.7
clang==5.0
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cryptography==35.0.0
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.24
daft==0.0.4
dask==2.12.0
datascience==0.10.6


In [ ]:
mkdir iris-classifier
mv iris-classifier.py iris-classifier


SyntaxError: ignored